# Welcome Notebook
This notebook walks through the process of creating and populating your first database with FinSpace Managed KX.

## Before you start
Before you start this notebook, it is assumed you have the following:
- FinSpace Managed KX environment created in AWS account
- S3 staging bucket for data and code
  - This notebook boto's profile and the Managed KX environment can access the bucket
- Setup in ~/.aws directory
  - config is set (json and region)
  - default credentials are set (aws_access_key_id, aws_secret_access_key, aws_session_token)

## Steps
1. Untar hdb.tar.gz for the hdb data
2. Upload hdb to staging S3 bucket
3. Create database
4. Add HDB data to database
5. Create a Cluster
6. Get the connectionString
7. Query Cluster using PyKX

## Managed kdb Insights Archtecture
<img src="images/Managed kdb Insights-HDB Migration.png"  width="60%">


In [3]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env import *

In [4]:
# Source data directory
SOURCE_DATA_DIR="hdb"

# S3 bucket for external data and code
S3_DEST=f"s3://{S3_BUCKET}/data/{SOURCE_DATA_DIR}/"
CODEBASE="code"
CODE_PATH=f"code/{CODEBASE}.zip"

NODE_COUNT=1
CACHE_SIZE=1200

# Managed KX Database and Cluster names to create
DB_NAME="welcomedb"
#DELETE_CLUSTER=True
#DELETE_DATABASE=True

create_delete=False

if create_delete:
    TODAY=datetime.datetime.now().strftime("%Y%m%d_%H%M")    
    DB_NAME=f"create_delete_db_{TODAY}"
    DELETE_CLUSTER=True
    DELETE_DATABASE=True

CLUSTER_NAME=f"cluster_{DB_NAME}"


In [5]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

# 0. Environment Check
Be sure the infrastructure ID has been entitled to the bucket you will be staging the HDB to. The environment will also need access to the KMX key used when creating the environment.

## Permission Templates

### S3 Permission
Example of code and data access to the same S3 bucket.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "finspace.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:GetObjectTagging",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::S3_BUCKET/*",
                "arn:aws:s3:::S3_BUCKET"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "ACCOUNT_ID"
                },
                "ArnEquals": {
                    "aws:SourceArn": "arn:aws:finspace:us-east-1:ACCOUNT_ID:kxEnvironment/ENV_ID/*"
                }
            }
        }
    ]
}

```

### KMS Key
Be sure the environment has access to use the KMS key given in environment creation.

```
"Statement": [
        {
            "Sid": "Enable Managed kdb Insights Access",
            "Effect": "Allow",
            "Principal": {
                "Service": "finspace.amazonaws.com"
            },
            "Action": [
                "kms:Encrypt",
                "kms:Decrypt",
                "kms:GenerateDataKey"
            ],
            "Resource": "arn:aws:kms:us-east-1:ACCOUNT_ID:key/KEY_ID",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "ACCOUNT_ID"
                },
                "ArnLike": {
                    "aws:SourceArn": "arn:aws:finspace:us-east-1:ACCOUNT_ID:kxEnvironment/ENV_ID/*"
                }
            }
        }
   ]
```


In [4]:
resp=get_kx_environment(client)

print("Environment Information")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))


Environment Information
{
    "availabilityZoneIds": [
        "use1-az1",
        "use1-az6",
        "use1-az2"
    ],
    "awsAccountId": "829845998889",
    "certificateAuthorityArn": "arn:aws:acm-pca:us-east-1:603187226814:certificate-authority/f9935247-b7f3-456c-b8e0-48b29b70c076",
    "creationTimestamp": "2023-06-05 20:44:00+00:00",
    "dedicatedServiceAccountId": "603187226814",
    "description": "Managed kdb Insights environment",
    "dnsStatus": "NONE",
    "environmentArn": "arn:aws:finspace:us-east-1:829845998889:kxEnvironment/jlcenjvtkgzrdek2qqv7ic",
    "environmentId": "jlcenjvtkgzrdek2qqv7ic",
    "kmsKeyId": "arn:aws:kms:us-east-1:829845998889:key/6643484a-28c3-46f4-8fba-87e67129e027",
    "name": "Managed_kdb_20230605",
    "status": "CREATED",
    "tgwStatus": "SUCCESSFULLY_UPDATED",
    "transitGatewayConfiguration": {
        "routableCIDRSpace": "100.64.0.0/26",
        "transitGatewayID": "tgw-0307e3bd926e58138"
    },
    "updateTimestamp": "2024-11-15 18:23

## 1. Untar hdb.tar.gz
hdb database will be found in hdb directory

In [5]:
!rm -rf hdb

In [6]:
!tar -xf hdb.tar.gz

In [7]:
!ls -la hdb

total 68
drwxrwxr-x 12 ec2-user ec2-user  4096 Nov 26 14:44 .
drwxrwxr-x  7 ec2-user ec2-user  4096 Nov 26 15:02 ..
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.16
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.17
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.18
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.19
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.20
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.21
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.22
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.23
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.24
drwxrwxr-x  3 ec2-user ec2-user  4096 Nov 26 14:44 2024.11.25
-rw-rw-r--  1 ec2-user ec2-user 16392 Nov 26 14:44 sym


# 2. Upload hdb data
using aws cli, copy hdb to staging bucket

In [8]:
cp = ""

if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}
"""

cp += f"""
aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
    
# execute the S3 copy
os.system(cp)

upload: hdb/2024.11.16/example/.d to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.16/example/.d
upload: hdb/2024.11.17/example/.d to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.17/example/.d
upload: hdb/2024.11.16/example/sym to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.16/example/sym
upload: hdb/2024.11.16/example/time to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.16/example/time
upload: hdb/2024.11.18/example/.d to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.18/example/.d
upload: hdb/2024.11.19/example/.d to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.19/example/.d
upload: hdb/2024.11.17/example/sym to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.17/example/sym
upload: hdb/2024.11.18/example/sym to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.18/example/sym
upload: hdb/2024.11.19/example/time to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.19/example/time
upload: hdb/2024.11.20/example/.d to s3://kdb-demo-829845998889-kms/data/hdb/2024.11.20/example

0

## 3. Create database

In [9]:
# assume it exists
create_db=False

try:
    resp = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
    resp.pop('ResponseMetadata', None)
except:
    # does not exist, will create
    create_db=True

if create_db:
    print(f"CREATING Database: {DB_NAME}")
    resp = client.create_kx_database(environmentId=ENV_ID, databaseName=DB_NAME, description="Welcome kdb database")
    resp.pop('ResponseMetadata', None)

    print(f"CREATED Database: {DB_NAME}")

print(json.dumps(resp,sort_keys=True,indent=4,default=str))

CREATING Database: welcomedb
CREATED Database: welcomedb
{
    "createdTimestamp": "2024-11-26 15:02:36.831000+00:00",
    "databaseArn": "arn:aws:finspace:us-east-1:829845998889:kxEnvironment/jlcenjvtkgzrdek2qqv7ic/kxDatabase/welcomedb",
    "databaseName": "welcomedb",
    "description": "Welcome kdb database",
    "environmentId": "jlcenjvtkgzrdek2qqv7ic",
    "lastModifiedTimestamp": "2024-11-26 15:02:36.831000+00:00"
}


## 4. Add HDB data to database

In [10]:
changes=[]

for f in os.listdir("hdb"):
    if os.path.isdir(f"hdb/{f}"):
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}/", 'dbPath': f"/{f}/" } )
    else:
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}", 'dbPath': f"/" } )

resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changeRequests=changes)

resp.pop('ResponseMetadata', None)
changeset_id = resp['changesetId']

print("Changeset...")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))

Changeset...
{
    "changeRequests": [
        {
            "changeType": "PUT",
            "dbPath": "/2024.11.19/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/hdb/2024.11.19/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2024.11.17/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/hdb/2024.11.17/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2024.11.25/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/hdb/2024.11.25/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2024.11.22/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/hdb/2024.11.22/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2024.11.23/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/hdb/2024.11.23/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2024.11.18/",
            "s3Path": "s3://kdb-demo-829

In [11]:
wait_for_changeset_status(client, environmentId=ENV_ID, databaseName=DB_NAME, changesetId=changeset_id, show_wait=True)

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:10, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:20, waiting 10 sec ...


{'changesetId': 'lsm0f3Z2MgmYnY89ZK2Mjw',
 'databaseName': 'welcomedb',
 'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'changeRequests': [{'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.19/',
   'dbPath': '/2024.11.19/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.17/',
   'dbPath': '/2024.11.17/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.25/',
   'dbPath': '/2024.11.25/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.22/',
   'dbPath': '/2024.11.22/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.23/',
   'dbPath': '/2024.11.23/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.18/',
   'dbPath': '/2024.11.18/'},
  {'changeType': 'PUT',
   's3Path': 's3://kdb-demo-829845998889-kms/data/hdb/2024.11.16/',
   'dbPath': '/2024.11.16/'},
  {'changeTy

In [12]:
note_str = ""

c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)

if len(c_set_list) == 0:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"  Changeset: {c_set_id}: Created: {c['createdTimestamp']} ({c['status']})")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: welcomedb, Changesets: 1 
  Changeset: lsm0f3Z2MgmYnY89ZK2Mjw: Created: 2024-11-26 15:02:37.550000+00:00 (COMPLETED)


changeType,s3Path,dbPath
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.19/,/2024.11.19/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.17/,/2024.11.17/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.25/,/2024.11.25/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.22/,/2024.11.22/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.23/,/2024.11.23/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.18/,/2024.11.18/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.16/,/2024.11.16/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.24/,/2024.11.24/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.21/,/2024.11.21/
PUT,s3://kdb-demo-829845998889-kms/data/hdb/2024.11.20/,/2024.11.20/


## 5. Create a Cluster for the database

In [13]:
# zip the code
#os.system(f"zip -r -X {CODEBASE}.zip {CODEBASE} -x '*.ipynb_checkpoints*'")
os.system(f"cd {CODEBASE}; zip -r -X ../{CODEBASE}.zip . -x '*.ipynb_checkpoints*' '.DS_Store';")

# copy code to S3
cp = ""

if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}
"""

cp += f"""
aws s3 cp  --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""

# execute the S3 copy
os.system(cp)

updating: init.q (deflated 35%)
updating: lib.q (stored 0%)
upload: ./code.zip to s3://kdb-demo-829845998889-kms/code/code.zip
2023-06-05 21:25:21          0 
2024-11-14 17:42:32      16585 basictick.zip
2024-11-15 18:24:29       1184 bmll.zip
2024-11-26 15:03:11        455 code.zip
2023-12-21 19:47:37        574 codebundle.zip
2024-02-02 21:34:56        582 codebundle1.zip
2023-12-21 21:26:00        582 codebundle2.zip
2024-11-14 17:30:52       2607 dbmaint.zip
2024-09-04 17:42:17        556 foo.q.zip
2023-11-22 14:58:53       1530 jpmc_code.zip
2024-01-01 19:57:08      33781 kdb-tick-flat-largetable.zip
2023-12-30 22:56:33      38867 kdb-tick-flat.zip
2024-01-08 13:05:33      28741 kdb-tick.zip
2023-08-22 16:58:18        765 qcode.zip
2024-10-16 22:31:45        465 taqcode.zip
2024-04-26 16:38:46     487423 torq_app.zip
2024-03-06 19:01:11    5807282 torq_app_20240306_1901.zip
2024-03-06 19:13:22    5807290 torq_app_20240306_1913.zip
2024-03-13 15:57:24    5807307 torq_app_20240313_1

0

In [6]:
print(f"Creating: {CLUSTER_NAME}")

resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterDescription=f"Demo Cluster for database {DB_NAME}",
    clusterType='HDB',
    releaseLabel = '1.0',
    capacityConfiguration={ "nodeType": "kx.s.large", "nodeCount": NODE_COUNT },
    databases=[{ 
        'databaseName': DB_NAME, 
        'cacheConfigurations': [
            {'dbPaths':['/'], 'cacheType': 'CACHE_250' }
        ] 
    }],
    cacheStorageConfigurations=[{ 'type': 'CACHE_250', 'size': CACHE_SIZE }],
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' },
    code={ 's3Bucket': S3_BUCKET, 's3Key': CODE_PATH },
    initializationScript=f"init.q",
    commandLineArguments=[
        {'key': 's', 'value': '2'}, 
        {'key': 'dbname', 'value': DB_NAME}, 
    ]
)

Creating: cluster_welcomedb


In [7]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: cluster_welcomedb status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wait 0:05:00, waiting 30 sec ...
Cluster: cluster_welcomedb status is CREATING, total wa

## 6. Get the Connection String
This assumes that the IAM role exists and the user (KDB_USERNAME) have beed already added as well.

In [8]:
try:
    resp = client.get_kx_cluster(environmentId=ENV_ID, clusterName=CLUSTER_NAME)
except client.exceptions.ResourceNotFoundException:
    print(F"Cluster: {CLUSTER_NAME} did not create")

if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
    sys.stderr.write("Error:\n {resp}")
else:
    resp.pop('ResponseMetadata', None)

kx_cluster = resp

print("Cluster: "+("-"*80))
print(json.dumps(kx_cluster, sort_keys=True, indent=4, default=str))


Cluster: --------------------------------------------------------------------------------
{
    "availabilityZoneId": "use1-az6",
    "azMode": "SINGLE",
    "cacheStorageConfigurations": [
        {
            "size": 1200,
            "type": "CACHE_250"
        }
    ],
    "capacityConfiguration": {
        "nodeCount": 1,
        "nodeType": "kx.s.large"
    },
    "clusterDescription": "Demo Cluster for database welcomedb",
    "clusterName": "cluster_welcomedb",
    "clusterType": "HDB",
    "code": {
        "s3Bucket": "kdb-demo-829845998889-kms",
        "s3Key": "code/code.zip"
    },
    "commandLineArguments": [
        {
            "key": "s",
            "value": "2"
        },
        {
            "key": "dbname",
            "value": "welcomedb"
        }
    ],
    "createdTimestamp": "2024-11-26 16:28:37.307000+00:00",
    "databases": [
        {
            "cacheConfigurations": [
                {
                    "cacheType": "CACHE_250",
                 

In [9]:
# Give permissions time to propogate after cluster creation....
time.sleep(60)


## 7. Query Cluster using PyKX

In [11]:
# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

In [12]:
# Tables
tables = hdb("tables[]").py()
print(f"Tables ({len(tables)}): {tables}")

# Schema
schema_pdf = hdb("meta `example").pd()
display(schema_pdf)


Tables (1): ['example']


,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
number,b'j',,


In [13]:
# Simple Query, uses function from lib
res_table = hdb("select counts:count i, avg_num: avg number, avg_sq_num: avg sq number by date from example").pd()
display(res_table)

# Number of Rows in Table
rows = hdb("count example").py()
print(f"Rows: {rows:,}")

,counts,avg_num,avg_sq_num
date,,,
2024-11-16,1000000,499800.464405,3.331963e+11
2024-11-17,1000000,499865.008159,3.332692e+11
2024-11-18,1000000,499912.379127,3.332060e+11
2024-11-19,1000000,500078.393386,3.334004e+11
2024-11-20,1000000,500264.684412,3.336606e+11
2024-11-21,1000000,499849.962912,3.333436e+11
2024-11-22,1000000,500082.257829,3.334472e+11
2024-11-23,1000000,500169.492354,3.336027e+11
2024-11-24,1000000,499903.047577,3.331833e+11


Rows: 10,000,000


## q Queries
You can also use the q magic to run q code directly on the remote cluster by connecting to the. cluster over IPC and sending code over to execute.

In [14]:
# get the connection string to the cluster
conn_str = get_kx_connection_string(client, 
                          environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# parse the connection string to components
host, port, username, password = parse_connection_string(conn_str)


In [15]:
%%q --host $host --port $port --user $username --pass $password

select counts:count i, avg_num: avg number, avg_sq_num: avg sq number by date from example

date      | counts  avg_num  avg_sq_num  
----------| -----------------------------
2024.11.16| 1000000 499800.5 3.331963e+11
2024.11.17| 1000000 499865   3.332692e+11
2024.11.18| 1000000 499912.4 3.33206e+11 
2024.11.19| 1000000 500078.4 3.334004e+11
2024.11.20| 1000000 500264.7 3.336606e+11
2024.11.21| 1000000 499850   3.333436e+11
2024.11.22| 1000000 500082.3 3.334472e+11
2024.11.23| 1000000 500169.5 3.336027e+11
2024.11.24| 1000000 499903   3.331833e+11
2024.11.25| 1000000 500277.5 3.335924e+11


# Clean Up

In [16]:
# Cluster Deletion
# ------------------------------------------------------------
db_list = list_kx_databases(client, environmentId=ENV_ID)
db_list

cluster_deleted=False

if DELETE_CLUSTER:   
    # list all clusters
    resp=client.get_kx_cluster(environmentId=ENV_ID, clusterName=CLUSTER_NAME)

    if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
        sys.stderr.write("Error:\n {resp}")
    else:
        resp.pop('ResponseMetadata', None)

    if resp['status'] != 'DELETING':
        try:
            resp = client.delete_kx_cluster(environmentId=ENV_ID, clusterName=CLUSTER_NAME)
            if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
                sys.stderr.write("Error:\n {resp}")
            else:
                resp.pop('ResponseMetadata', None)
        except Exception as e: 
            sys.stderr.write(f"Error deleting cluster: {CLUSTER_NAME}\n{e}")
            cluster_deleted = False

    try:
        wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, status='DELETED', show_wait=False)
        print()
        print("** DONE **")

        cluster_deleted = True
    except client.exceptions.ResourceNotFoundException:
        cluster_deleted = True
else:
    print(f"DELETE_CLUSTER: {DELETE_CLUSTER}")

NameError: name 'DELETE_CLUSTER' is not defined

In [ ]:
# Database Deletion
# Requires cluster to have been deleted
if DELETE_DATABASE:
    if cluster_deleted:
        # if the database exists, delete it
        if has_database(client, environmentId=ENV_ID, databaseName=DB_NAME):
            try:
                resp = client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
                if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
                    sys.stderr.write("Error:\n {resp}")
                else:
                    resp.pop('ResponseMetadata', None)

                resp
            except Exception as e: 
                sys.stderr.write(f"Error: \n{e}")
        else:
            print(f"Database already deleted: {DB_NAME} ")
    else:
        print(f"Cluster deleted? {cluster_deleted}, will not delete database if cluster not deleted")
else:
    print(f"DELETE_DATABASE: {DELETE_DATABASE}")

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )